In [7]:
import pandas as pd

# Load the CSV file
train_df = pd.read_csv('train.csv')

# Explore the dataset
print(train_df.head())
print(train_df.shape)
print(train_df['diagnosis'].value_counts())

        id_code  diagnosis
0  000c1434d8d7          2
1  001639a390f0          4
2  0024cdab0c1e          1
3  002c21358ce6          0
4  005b95c28852          0
(3662, 2)
0    1805
2     999
1     370
4     295
3     193
Name: diagnosis, dtype: int64


In [10]:
import tensorflow as tf
import numpy as np
import os
from prettytable import PrettyTable
import pickle
import multiprocessing
from multiprocessing.pool import ThreadPool
print(multiprocessing.cpu_count()," CPU cores")

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False

from sklearn.metrics import confusion_matrix, cohen_kappa_score,accuracy_score

from PIL import Image
import cv2

from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers,Model,Sequential
from tensorflow.keras.layers import Input,GlobalAveragePooling2D,Dropout,Dense,Activation
# from keras.callbacks.callbacks import EarlyStopping,ReduceLROnPlateau



12  CPU cores


In [11]:
import os
import cv2
import numpy as np

# Set the image size
IMG_SIZE = 512

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

def circle_crop(img, sigmaX = 30):   
    """
    Create circular crop around image centre    
    """    
    img = crop_image_from_gray(img)    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img=cv2.addWeighted(img,4, cv2.GaussianBlur( img , (0,0) , sigmaX) ,-4 ,128)
    return img 

def preprocess_image(file):
    input_filepath = os.path.join('./','train_images_resized','{}.png'.format(file))
    output_filepath = os.path.join('./','train_images_resized_preprocessed','{}.png'.format(file))
    
    img = cv2.imread(input_filepath)
    img = circle_crop(img) 
    cv2.imwrite(output_filepath, cv2.resize(img, (IMG_SIZE,IMG_SIZE)))

def multiprocess_image_processor(process:int, imgs:list):
    """
    Inputs:
        process: (int) number of process to run
        imgs:(list) list of images
    """
    print(f'MESSAGE: Running {process} process')
    results = ThreadPool(process).map(preprocess_image, imgs)
    return results

In [14]:
existing_files = 0
missing_files = 0
image_dir = 'train_images'

# Iterate through the 'id_code' values in the DataFrame
for file_name in train_df['id_code']:
    file_path = os.path.join(image_dir, f"{file_name}.png")
    
    # Check if the file exists
    if os.path.exists(file_path):
        existing_files += 1
    else:
        missing_files += 1

print(f"Number of existing image files: {existing_files}")
print(f"Number of missing image files: {missing_files}")

Number of existing image files: 3662
Number of missing image files: 0


In [20]:
import os
import cv2
import numpy as np
import pandas as pd
from multiprocessing.pool import ThreadPool

# Set the image size
IMG_SIZE = 512

def crop_image_from_gray(img, tol=7):
    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1), mask.any(0))]
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0):  # image is too dark so that we crop out everything,
            return img  # return original image
        else:
            img1 = img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2 = img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3 = img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1, img2, img3], axis=-1)
        return img

def circle_crop(img, sigmaX=30):
    img = crop_image_from_gray(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width, depth = img.shape
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img = cv2.addWeighted(img, 4, cv2.GaussianBlur(img, (0,0), sigmaX), -4, 128)
    return img

def preprocess_image(file):
    input_filepath = os.path.join('train_images_resized', f"{file}.png")
    output_filepath = os.path.join('train_images_resized_preprocessed', f"{file}.png")
    
    try:
        img = cv2.imread(input_filepath)
        if img is not None:
            img = circle_crop(img)
            cv2.imwrite(output_filepath, cv2.resize(img, (IMG_SIZE,IMG_SIZE)))
        else:
            print(f"Error loading image: {file}.png")
            return None
    except Exception as e:
        print(f"Error processing image: {file}.png")
        print(e)
        return None

def multiprocess_image_processor(imgs):
    """
    Inputs:
        imgs:(list) list of images
    """
    # Get the number of CPU cores
    num_processes = os.cpu_count()
    
    print(f'MESSAGE: Running {num_processes} processes')
    
    # Create a pool of worker processes
    pool = ThreadPool(num_processes)
    
    # Apply the preprocess_image function to each image in parallel
    results = pool.map(preprocess_image, imgs)
    
    # Close the pool and wait for the processes to finish
    pool.close()
    pool.join()
    
    return results


In [21]:
def image_resize_save(file):
    input_filepath = os.path.join('train_images', f"{file}.png")
    output_filepath = os.path.join('train_images_resized', f"{file}.png")
    img = cv2.imread(input_filepath)
    cv2.imwrite(output_filepath, cv2.resize(img, (IMG_SIZE,IMG_SIZE)))

def multiprocess_image_downloader(imgs):
    """
    Inputs:
        imgs:(list) list of images
    """
    # Get the number of CPU cores
    num_processes = os.cpu_count()
    
    print(f'MESSAGE: Running {num_processes} processes')
    
    # Create a pool of worker processes
    pool = ThreadPool(num_processes)
    
    # Apply the image_resize_save function to each image in parallel
    results = pool.map(image_resize_save, imgs)
    
    # Close the pool and wait for the processes to finish
    pool.close()
    pool.join()
    
    return results

In [22]:
# Perform train-test split
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(train_df, test_size=0.2, random_state=42)

In [26]:
os.makedirs('train_images_resized', exist_ok=True)
os.makedirs('train_images_resized_preprocessed', exist_ok=True)
os.makedirs('test_images_resized', exist_ok=True)
os.makedirs('test_images_resized_preprocessed', exist_ok=True)

In [27]:
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [28]:
# Resize train images
multiprocess_image_downloader(list(df_train.id_code.values))

# Resize test images
multiprocess_image_downloader(list(df_test.id_code.values))

# Preprocess train images
multiprocess_image_processor(list(df_train.id_code.values))

# Preprocess test images
multiprocess_image_processor(list(df_test.id_code.values))

MESSAGE: Running 12 processes
MESSAGE: Running 12 processes
MESSAGE: Running 12 processes
MESSAGE: Running 12 processes


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [29]:
train_df.shape

(2929, 2)

In [30]:
test_df.shape

(733, 2)

In [31]:
print(test_df['diagnosis'].value_counts())

0    351
2    213
1     68
4     65
3     36
Name: diagnosis, dtype: int64
